# Wind Data Combiner

Este notebook combina os dados de três arquivos de vento (Dir.txt, W.txt, WS.txt) em um único dataset regularizado.

In [30]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso")

Bibliotecas importadas com sucesso


In [31]:
# Definir caminhos dos arquivos
raw_data_path = Path('../raw_data')
output_path = Path('../')

# Lista dos arquivos a serem processados
files_to_process = ['Dir.txt', 'W.txt', 'WS.txt']

print(f"Diretório de dados brutos: {raw_data_path}")
print(f"Diretório de saída: {output_path}")
print(f"Arquivos a processar: {files_to_process}")

Diretório de dados brutos: ../raw_data
Diretório de saída: ..
Arquivos a processar: ['Dir.txt', 'W.txt', 'WS.txt']


In [32]:
# Função para ler e processar cada arquivo
def read_wind_data(file_path, prefix):
    """
    Lê arquivo de dados de vento e renomeia colunas com prefixo.
    
    Args:
        file_path: Caminho para o arquivo
        prefix: Prefixo para as colunas (Dir, W, WS)
    
    Returns:
        DataFrame com colunas renomeadas
    """
    df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    
    # Converter coluna DT para datetime
    df['DT'] = pd.to_datetime(df['DT'])
    
    # Renomear colunas (exceto DT) com o prefixo
    column_mapping = {col: f"{prefix}-{col}" for col in df.columns if col != 'DT'}
    df = df.rename(columns=column_mapping)
    
    print(f"Arquivo {file_path.name} carregado: {len(df)} registros")
    print(f"Período: {df['DT'].min()} até {df['DT'].max()}")
    
    return df

print("Função de leitura definida")

Função de leitura definida


In [33]:
# Carregar todos os arquivos
dataframes = []

for file_name in files_to_process:
    file_path = raw_data_path / file_name
    prefix = file_name.replace('.txt', '')  # Remove extensão para usar como prefixo
    
    if file_path.exists():
        df = read_wind_data(file_path, prefix)
        dataframes.append(df)
        print(f"✓ {file_name} processado")
    else:
        print(f"✗ Arquivo {file_name} não encontrado")

print(f"\nTotal de arquivos carregados: {len(dataframes)}")

Arquivo Dir.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ Dir.txt processado
Arquivo W.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ W.txt processado
Arquivo WS.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ WS.txt processado

Total de arquivos carregados: 3
Arquivo WS.txt carregado: 41786 registros
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ WS.txt processado

Total de arquivos carregados: 3


In [34]:
# Combinar todos os DataFrames usando merge na coluna DT
if len(dataframes) > 0:
    # Começar com o primeiro DataFrame
    combined_df = dataframes[0]
    
    # Fazer merge com os demais
    for df in dataframes[1:]:
        combined_df = pd.merge(combined_df, df, on='DT', how='outer')
    
    print(f"Dataset combinado criado com {len(combined_df)} registros")
    print(f"Colunas no dataset: {len(combined_df.columns)}")
    print(f"Primeiras colunas: {list(combined_df.columns[:10])}")
else:
    print("Nenhum arquivo foi carregado")

Dataset combinado criado com 42524 registros
Colunas no dataset: 79
Primeiras colunas: ['DT', 'Dir-SODAR-30', 'Dir-SODAR-40', 'Dir-SODAR-50', 'Dir-SODAR-60', 'Dir-SODAR-70', 'Dir-SODAR-80', 'Dir-SODAR-90', 'Dir-SODAR-100', 'Dir-SODAR-110']


In [35]:
# Filtrar apenas as colunas de SODAR-40 até SODAR-140
# Primeiro renomear DT para timestamp
combined_df = combined_df.rename(columns={'DT': 'timestamp'})

# Selecionar colunas timestamp e SODAR-40 até SODAR-140
sodar_columns = ['timestamp']
for height in range(40, 150, 10):  # 40, 50, 60, ..., 140
    for prefix in ['Dir', 'W', 'WS']:
        col_name = f"{prefix}-SODAR-{height}"
        if col_name in combined_df.columns:
            sodar_columns.append(col_name)

# Criar DataFrame filtrado
wind_data = combined_df[sodar_columns].copy()

print(f"Dataset filtrado criado com {len(wind_data)} registros")
print(f"Colunas selecionadas: {len(wind_data.columns) - 1} (excluindo timestamp)")
print(f"Alturas SODAR incluídas: 40m até 140m")

Dataset filtrado criado com 42524 registros
Colunas selecionadas: 33 (excluindo timestamp)
Alturas SODAR incluídas: 40m até 140m


In [36]:
# Verificar estrutura atual dos dados
print("=== Estrutura do Dataset Atual ===")
print(f"Forma: {wind_data.shape}")
print(f"\nPrimeiros e últimos timestamps:")
print(f"Início: {wind_data['timestamp'].min()}")
print(f"Fim: {wind_data['timestamp'].max()}")
print(f"\nVerificação de valores nulos por coluna:")
print(wind_data.isnull().sum().head(10))

=== Estrutura do Dataset Atual ===
Forma: (42524, 34)

Primeiros e últimos timestamps:
Início: 2021-11-10 17:20:00
Fim: 2022-10-09 08:30:00

Verificação de valores nulos por coluna:
timestamp         0
Dir-SODAR-40    487
W-SODAR-40      434
WS-SODAR-40     487
Dir-SODAR-50    474
W-SODAR-50      489
WS-SODAR-50     474
Dir-SODAR-60    486
W-SODAR-60      536
WS-SODAR-60     486
dtype: int64


In [37]:
# Regularizar série temporal - criar índice completo com intervalos de 10 minutos
print("=== Regularização da Série Temporal ===")

# Ordenar por timestamp
wind_data = wind_data.sort_values('timestamp')

# Remover duplicatas de timestamp
duplicates_before = wind_data.duplicated(subset=['timestamp']).sum()
wind_data = wind_data.drop_duplicates(subset=['timestamp'])
print(f"Duplicatas removidas: {duplicates_before}")

# Criar série temporal completa com intervalos de 10 minutos
start_time = wind_data['timestamp'].min()
end_time = wind_data['timestamp'].max()

# Gerar todos os timestamps de 10 em 10 minutos
complete_timeline = pd.date_range(start=start_time, end=end_time, freq='10min')
complete_df = pd.DataFrame({'timestamp': complete_timeline})

print(f"Período original: {start_time} até {end_time}")
print(f"Registros originais: {len(wind_data)}")
print(f"Timestamps esperados (10min): {len(complete_timeline)}")
print(f"Timestamps faltantes: {len(complete_timeline) - len(wind_data)}")

=== Regularização da Série Temporal ===
Duplicatas removidas: 833
Período original: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
Registros originais: 41691
Timestamps esperados (10min): 47900
Timestamps faltantes: 6209


In [38]:
# Fazer merge para preencher timestamps faltantes com NaN
wind_data_regularized = pd.merge(complete_df, wind_data, on='timestamp', how='left')

print("=== Dataset Regularizado ===")
print(f"Forma final: {wind_data_regularized.shape}")
print(f"Total de timestamps: {len(wind_data_regularized)}")
print(f"Timestamps únicos: {wind_data_regularized['timestamp'].nunique()}")

# Verificar se há duplicatas finais
final_duplicates = wind_data_regularized.duplicated(subset=['timestamp']).sum()
print(f"Duplicatas finais: {final_duplicates}")

# Estatísticas de valores faltantes
missing_stats = wind_data_regularized.isnull().sum()
print(f"\nEstatísticas de valores faltantes:")
print(f"Colunas com dados faltantes: {(missing_stats > 0).sum()}")
print(f"Total de valores NaN: {missing_stats.sum()}")

=== Dataset Regularizado ===
Forma final: (47900, 34)
Total de timestamps: 47900
Timestamps únicos: 47900
Duplicatas finais: 0

Estatísticas de valores faltantes:
Colunas com dados faltantes: 33
Total de valores NaN: 233444


In [39]:
# Reorganizar e renomear colunas conforme especificado
print("=== Reorganização e Renomeação das Colunas ===")

# Primeiro, criar mapeamento para renomear colunas
column_mapping = {}
for height in range(40, 150, 10):  # 40, 50, 60, ..., 140
    # WS-SODAR-XX -> wsXX
    old_ws = f"WS-SODAR-{height}"
    new_ws = f"ws{height}"
    if old_ws in wind_data_regularized.columns:
        column_mapping[old_ws] = new_ws
    
    # W-SODAR-XX -> vXX (vento vertical)
    old_w = f"W-SODAR-{height}"
    new_w = f"v{height}"
    if old_w in wind_data_regularized.columns:
        column_mapping[old_w] = new_w
    
    # Dir-SODAR-XX -> dirXX
    old_dir = f"Dir-SODAR-{height}"
    new_dir = f"dir{height}"
    if old_dir in wind_data_regularized.columns:
        column_mapping[old_dir] = new_dir

# Aplicar renomeação
wind_data_regularized = wind_data_regularized.rename(columns=column_mapping)

print(f"Colunas renomeadas: {len(column_mapping)}")

# Organizar ordem das colunas: timestamp, ws40-ws140, v40-v140, dir40-dir140
ordered_columns = ['timestamp']

# Adicionar colunas WS (velocidade do vento)
for height in range(40, 150, 10):
    ws_col = f"ws{height}"
    if ws_col in wind_data_regularized.columns:
        ordered_columns.append(ws_col)

# Adicionar colunas W (vento vertical)
for height in range(40, 150, 10):
    v_col = f"v{height}"
    if v_col in wind_data_regularized.columns:
        ordered_columns.append(v_col)

# Adicionar colunas Dir (direção)
for height in range(40, 150, 10):
    dir_col = f"dir{height}"
    if dir_col in wind_data_regularized.columns:
        ordered_columns.append(dir_col)

# Reordenar o DataFrame
wind_data_regularized = wind_data_regularized[ordered_columns]

print(f"Ordem das colunas reorganizada:")
print(f"  - Velocidade (WS): ws40 até ws140")
print(f"  - Vento vertical (W): v40 até v140") 
print(f"  - Direção (Dir): dir40 até dir140")
print(f"Total de colunas ordenadas: {len(ordered_columns)}")

=== Reorganização e Renomeação das Colunas ===
Colunas renomeadas: 33
Ordem das colunas reorganizada:
  - Velocidade (WS): ws40 até ws140
  - Vento vertical (W): v40 até v140
  - Direção (Dir): dir40 até dir140
Total de colunas ordenadas: 34


In [40]:
# Exibir amostra dos dados finais
print("=== Amostra do Dataset Final ===")
print("Primeiras 5 linhas:")
print(wind_data_regularized.head())
print("\nÚltimas 5 linhas:")
print(wind_data_regularized.tail())
print(f"\nNomes das colunas reorganizadas:")
print(f"Colunas: {list(wind_data_regularized.columns)}")
print("\nInfo do dataset:")
print(wind_data_regularized.info())

=== Amostra do Dataset Final ===
Primeiras 5 linhas:
            timestamp  ws40  ws50  ws60  ws70  ws80  ws90  ws100  ws110  \
0 2021-11-10 17:20:00  5.03  5.53  5.54  5.66  5.74  5.82   5.83   5.79   
1 2021-11-10 17:30:00  4.94  5.30  5.29  5.72  5.87  5.98   5.99   6.06   
2 2021-11-10 17:40:00  4.44  5.96  5.66  6.44  6.55  6.61   6.67   6.67   
3 2021-11-10 17:50:00  4.28  4.91  5.04  5.80  5.95  6.03   6.15   6.17   
4 2021-11-10 18:00:00  4.53  5.02  4.66  5.52  5.62  5.70   5.90   5.93   

   ws120  ...  dir50  dir60  dir70  dir80  dir90  dir100  dir110  dir120  \
0   5.71  ...   30.5   29.2   31.4   31.3   32.3    33.3    33.6    32.8   
1   6.18  ...   37.7   35.0   34.7   34.8   35.6    36.4    37.6    38.2   
2   6.63  ...   42.7   41.3   42.0   43.1   43.0    43.3    42.9    43.2   
3   6.24  ...   44.3   45.7   43.7   45.0   45.7    46.8    46.2    46.3   
4   6.15  ...   49.5   44.6   43.6   45.1   44.4    42.5    40.2    39.0   

   dir130  dir140  
0    31.4    30.8  

In [41]:
# Salvar o dataset regularizado
output_file = output_path / 'wind_data.csv'

# Salvar o arquivo
wind_data_regularized.to_csv(output_file, index=False, encoding='utf-8')

print(f"=== Dataset Salvo ===")
print(f"Arquivo: {output_file}")
print(f"Tamanho: {len(wind_data_regularized)} registros")
print(f"Colunas: {len(wind_data_regularized.columns)}")
print(f"Período: {wind_data_regularized['timestamp'].min()} até {wind_data_regularized['timestamp'].max()}")

# Verificar se arquivo foi criado
if output_file.exists():
    file_size = output_file.stat().st_size / (1024 * 1024)  # MB
    print(f"✓ Arquivo criado com sucesso ({file_size:.2f} MB)")
else:
    print("✗ Erro ao criar arquivo")

=== Dataset Salvo ===
Arquivo: ../wind_data.csv
Tamanho: 47900 registros
Colunas: 34
Período: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
✓ Arquivo criado com sucesso (7.75 MB)


In [42]:
# Resumo final do processamento
print("=== RESUMO DO PROCESSAMENTO ===")
print(f"\n1. Arquivos processados: {', '.join(files_to_process)}")
print(f"2. Dataset combinado: wind_data.csv")
print(f"3. Período de dados: {wind_data_regularized['timestamp'].min()} até {wind_data_regularized['timestamp'].max()}")
print(f"4. Intervalo: 10 minutos")
print(f"5. Total de registros: {len(wind_data_regularized):,}")
print(f"6. Colunas de dados: {len(wind_data_regularized.columns) - 1} (alturas 40m até 140m)")
print(f"7. Duplicatas removidas: {duplicates_before}")
print(f"8. Timestamps adicionados: {len(complete_timeline) - len(wind_data) + duplicates_before}")
print(f"\n9. Reorganização das colunas:")
print(f"   - Velocidade do vento (WS): ws40, ws50, ..., ws140")
print(f"   - Vento vertical (W): v40, v50, ..., v140")
print(f"   - Direção (Dir): dir40, dir50, ..., dir140")
print(f"10. Renomeação aplicada:")
print(f"   - 'WS-SODAR-XX' → 'wsXX'")
print(f"   - 'W-SODAR-XX' → 'vXX'")
print(f"   - 'Dir-SODAR-XX' → 'dirXX'")
print(f"\n✓ Processamento concluído com sucesso!")

=== RESUMO DO PROCESSAMENTO ===

1. Arquivos processados: Dir.txt, W.txt, WS.txt
2. Dataset combinado: wind_data.csv
3. Período de dados: 2021-11-10 17:20:00 até 2022-10-09 08:30:00
4. Intervalo: 10 minutos
5. Total de registros: 47,900
6. Colunas de dados: 33 (alturas 40m até 140m)
7. Duplicatas removidas: 833
8. Timestamps adicionados: 7042

9. Reorganização das colunas:
   - Velocidade do vento (WS): ws40, ws50, ..., ws140
   - Vento vertical (W): v40, v50, ..., v140
   - Direção (Dir): dir40, dir50, ..., dir140
10. Renomeação aplicada:
   - 'WS-SODAR-XX' → 'wsXX'
   - 'W-SODAR-XX' → 'vXX'
   - 'Dir-SODAR-XX' → 'dirXX'

✓ Processamento concluído com sucesso!
